In [30]:
# Install required packages (run this cell first if packages are not installed)
%pip install firebase-admin google-cloud-storage python-dotenv pandas


Defaulting to user installation because normal site-packages is not writeable
  Using cached firebase_admin-7.1.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached google_cloud_storage-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached cachecontrol-0.14.4-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached google_cloud_firestore-2.21.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached h2-4.3.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached h11-0.16.0-py3-none-any.whl


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\deepa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables from .env file
# Explicitly specify the path to ensure it's found
env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(env_path)

# Verify that environment variables are loaded
print("Environment variables loaded from:", env_path)
print("FIREBASE_PROJECT_ID:", "✓ SET" if os.getenv('FIREBASE_PROJECT_ID') else "✗ NOT SET")
print("FIREBASE_CLIENT_EMAIL:", "✓ SET" if os.getenv('FIREBASE_CLIENT_EMAIL') else "✗ NOT SET")

Environment variables loaded from: e:\coffee_project\coffee_shop_customer_service_chatbot-main\python_code\.env
FIREBASE_PROJECT_ID: ✓ SET
FIREBASE_CLIENT_EMAIL: ✓ SET


# Firebase Init

In [34]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }
  



In [35]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'coffee-shop-app-62910.firebasestorage.app',
     'databaseURL': 'https://coffee-shop-app-62910-default-rtdb.firebaseio.com/'
})

In [36]:
bucket = storage.bucket()

# Upload Data

In [37]:
image_folder_path = './products/images/'

In [38]:
products_collection = db.reference('products')

In [39]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [40]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [41]:
for index, row in df.iterrows():
    print(index, row['name'])
    
    image_path = os.path.join(image_folder_path,row['image_path'])
    
    image_url = upload_image(bucket,image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url']= image_url
    
    # Add to Firestore
    products_collection.push().set(product_data)
    

0 Cappuccino
1 Jumbo Savory Scone
2 Latte
3 Chocolate Chip Biscotti
4 Espresso shot
5 Hazelnut Biscotti
6 Chocolate Croissant
7 Dark chocolate
8 Cranberry Scone
9 Croissant
10 Almond Croissant
11 Ginger Biscotti
12 Oatmeal Scone
13 Ginger Scone
14 Chocolate syrup
15 Hazelnut syrup
16 Carmel syrup
17 Sugar Free Vanilla syrup
